In [1]:
from Feature_Selection import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold

import joblib
import time

In [2]:
X = joblib.load('./AML_data/meth.pkl')

phenodf = joblib.load('./AML_data/pheno.pkl')

In [3]:
phenodf

,sample.type,FAB,genotype,relapse
public_id,,,,
AML_001,diagnostic,M2,normal,True
AML_002,diagnostic,M5,t(11;19),True
AML_003,diagnostic,M1,normal,False
AML_004_r,relapse,M5,NaN,True
AML_005,diagnostic,M2,mono 7,True
...,...,...,...,...
AML_122,diagnostic,M2,normal,False
AML_033,diagnostic,M6,normal,True
AML_123,diagnostic,M4,inv(16),False


In [4]:
# Create a new column for genotypes to merge some groups together

# Nas, No result and other will form one group
mll = ['other 11q23/MLL', 't(9;11)', 't(10;11)','t(11;19)']
other = ['normal', 'mono 7', 'inv(16)','other clon abn', '3q21q26', 't(8;21)', 'sole+8', 't(15;17)']

In [5]:
finalgenotype = []

for data in phenodf.genotype:

    if data in mll:
        finalgenotype.append('MLL rearranged')
        
    elif data == 'no result':
        finalgenotype.append('No result')
        
    elif data in other:
        finalgenotype.append(data)
        
    else:
        finalgenotype.append('No result')
        

In [6]:
phenodf['finalgenotype'] = finalgenotype

In [7]:
phenodf

,sample.type,FAB,genotype,relapse,finalgenotype
public_id,,,,,
AML_001,diagnostic,M2,normal,True,normal
AML_002,diagnostic,M5,t(11;19),True,MLL rearranged
AML_003,diagnostic,M1,normal,False,normal
AML_004_r,relapse,M5,NaN,True,No result
AML_005,diagnostic,M2,mono 7,True,mono 7
...,...,...,...,...,...
AML_122,diagnostic,M2,normal,False,normal
AML_033,diagnostic,M6,normal,True,normal
AML_123,diagnostic,M4,inv(16),False,inv(16)


In [8]:
X.shape

(142, 406830)

In [9]:
# Filter out the 43 undefined samples

In [10]:
phenodf = phenodf[~phenodf['finalgenotype'].isin(['No result', 'other clon abn'])]

In [11]:
X = X[X.index.get_level_values(1).isin(phenodf.index)]

In [12]:
# Create cross a validation loop

In [13]:
cv_outer = KFold(n_splits=5, shuffle=True, random_state=1)

In [14]:
indices_list = []

pca_indices = []
lvhc_indices = []
i = 1
for train_ix, val_ix in cv_outer.split(X):
    print('---------------------------------')
    print('---------------------------------')
    
    print('Fold {}'.format(i))
    print('---------------------------')
    # split data
    X_train, X_val = X.iloc[train_ix, :], X.iloc[val_ix, :]
    

    #Imputation of missing values
    #Impute inputs
    imp = SimpleImputer(missing_values=np.nan, strategy='median')
    imp.fit(X_train)
    X_train = pd.DataFrame(imp.transform(X_train), columns = X_train.columns)
    X_val = pd.DataFrame(imp.transform(X_val), columns = X_val.columns)

    
    # Double feature selection on the train set and then apply the indices to both datasets
    # PCA contribution selection
    pcaindices = PC_contribution(X_train, 15, 10) # 10 x 2 CpGs per Component for first 15 Components

    # Low variance high correlation selection

    X_tr, X_v = var_corr_sel(X_train, X_val, 0.1, 0.7) #it will remove low variance and high correlated probes
    lvhcindices =  X_tr.columns.to_list()

    unionindices = sorted(set(pcaindices).union(lvhcindices))
    print('{} features have been selected after feature selection for fold {}'.format(len(unionindices), i))
    print('---------------------------------------------------------------------------')
    print('{} features have been selected after PCA-based FS for fold {}'.format(len(pcaindices), i))
    print('{} features have been selected after LVHC selection for fold {}'.format(len(lvhcindices), i))
    X_train = X_train[unionindices]
    X_val = X_val[unionindices]

    indices_list.extend(unionindices) # extend this list to get all unique probes during all folds
    pca_indices.extend(pcaindices)
    lvhc_indices.extend(lvhcindices)
    i += 1   

---------------------------------
---------------------------------
Fold 1
---------------------------
515 features have been selected after feature selection for fold 1
---------------------------------------------------------------------------
296 features have been selected after PCA-based FS for fold 1
240 features have been selected after LVHC selection for fold 1
---------------------------------
---------------------------------
Fold 2
---------------------------
507 features have been selected after feature selection for fold 2
---------------------------------------------------------------------------
299 features have been selected after PCA-based FS for fold 2
225 features have been selected after LVHC selection for fold 2
---------------------------------
---------------------------------
Fold 3
---------------------------
470 features have been selected after feature selection for fold 3
---------------------------------------------------------------------------
295 featur

In [15]:
finalindices = np.unique(indices_list)

In [16]:
len(finalindices)

1300

In [17]:
len(np.unique(pca_indices))

955

In [18]:
len(np.unique(lvhc_indices))

456

In [19]:
print(len(sorted(set(np.unique(pca_indices)).union(np.unique(lvhc_indices)))))

1300


In [20]:
joblib.dump(finalindices, './AML_data/unionindices.pkl') 

['unionindices.pkl']